In [8]:

from multiprocessing import Pool
import numpy as np
import pandas as pd
import queue
import os

from Backtest.strategy import Strategy
from Backtest.event import EventType
from Backtest.backtest import Backtest
from Backtest.data import OHLCDataHandler
from Backtest.portfolio import PortfolioHandler
from Backtest.open_json_gz_files import open_json_gz_files
from Backtest.generate_bars import generate_bars


In [9]:
config = {
    "csv_dir": "C:/backtest/Binance",
    "out_dir": "C:/backtest/results/MOMStrategy",
    "title": "MOMStrategy",
    "is_plot": True,
    "save_plot": True,
    "save_tradelog": True,
    "start_date": pd.Timestamp("2018-01-01T00:0:00", freq="60" + "T"),  # str(freq) + "T"
    "end_date": pd.Timestamp("2018-09-01T00:00:00", freq="60" + "T"),
    "equity": 1.0,
    "freq": 60,  # min
    "commission_ratio": 0.002,
    "suggested_quantity": None,     # None or a value
    "max_quantity": None,           # None or a value, Maximum purchase quantity
    "min_quantity": None,           # None or a value, Minimum purchase quantity
    "min_handheld_cash": None,      # None or a value, Minimum handheld funds
    "exchange": "Binance",
    "tickers": ['BTCUSDT', 'CMTBNB', 'CMTBTC', 'CMTETH', 
                'ETHUSDT', 'LTCUSDT', 'VENBNB', #  'EOSUSDT' 'XRPUSDT'
               'VENBTC', 'VENETH']
}

In [10]:
# trading_data = {}
# for ticker in config['tickers']:
#     # trading_data[ticker] = open_gz_files(config['csv_dir'], ticker)
#     trading_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '.h5', key=ticker)

ohlc_data = {}
for ticker in config['tickers']:
    # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
    ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker +'_OHLC_60min.h5', key=ticker)

trading_data = None

In [11]:
class MOMStrategy(Strategy):
    def __init__(self, config, events, data_handler, portfolio_handler,
                 window = 10, num = 3, rev = 0):  
        # rev = 0: buy first num, rev = 1: buy last num
        self.config = config
        self.data_handler = data_handler
        self.portfolio_handler = portfolio_handler
        self.tickers = self.config['tickers']
        self.events = events
        self.holdinds = self._calculate_initial_holdings()
        
        self.bar_date = None
        self.window = window
        self.num = num
        self.rev = rev
        self.MOM_tickers = {}
        self.REV_tickers = {}

    def _calculate_initial_holdings(self):
        holdings = {}
        for s in self.tickers:
            holdings[s] = "EMPTY"
        return holdings
    
    def generate_signals(self, event):
        if event.type == EventType.MARKET:
            bar_date = event.timestamp
            if self.bar_date == bar_date:
                pass
            else:
                self.bar_date = bar_date
                bars = {}
                for ticker in self.tickers:
                    bars[ticker] = self.data_handler.get_latest_bars_values(
                                    ticker, "close", N=self.window
                                    )
                bars = pd.DataFrame(bars)

                change = (bars.iloc[-1,:] - bars.iloc[0,:]) / bars.iloc[0,:]
                change = change.sort_values(ascending= False)
                
                current_tickers = self.portfolio_handler.current_tickers
                cash_for_order = self.portfolio_handler.cash_for_order
                
                
                MOM_tickers = change.iloc[:self.num].keys()
                REV_tickers = change.iloc[-self.num:].keys()
                self.MOM_tickers[bar_date] = MOM_tickers
                self.REV_tickers[bar_date] = REV_tickers
                
                
                if self.rev == 0:
                    
                    for ticker in current_tickers:
                        if ticker not in MOM_tickers:
                            self.generate_sell_signals(ticker, bar_date, "SHORT")
                            quantity = self.portfolio_handler.current_tickers_info[ticker]['quantity']
                            event_price = bars.loc[len(bars) - 1, ticker]
                            cash_for_order += (event_price * quantity) * (1-self.portfolio_handler.commission_ratio)
                    
                    MOM_num = sum([ticker not in current_tickers for ticker in MOM_tickers])
                    if MOM_num != 0:
                        suggested_cash = cash_for_order / MOM_num
                        for ticker in MOM_tickers:
                            if ticker not in current_tickers:
                                self.generate_buy_signals(ticker, bar_date, "LONG", suggested_cash = suggested_cash)
                    
                elif self.rev == 1:
                    REV_tickers = change.iloc[-self.num:].keys()
                    REV_num = sum([ticker in self.portfolio_handler.current_tickers for ticker in REV_tickers])
                    pass

In [12]:
events_queue = queue.Queue()

In [13]:
data_handler = OHLCDataHandler(
    config, events_queue,
    trading_data = trading_data, ohlc_data = ohlc_data
)

Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-08-07 23:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-08-07 23:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-08-07 23:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-08-07 23:00:00
Data Time Interval for ETHUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-08-07 23:00:00
Data Time Interval for LTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-08-07 23:00:00
Data Time Interval for VENBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-07-23 03:00:00
Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-07-23 03:00:00
Data Time Interval for VENETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-07-23 03:00:00


In [14]:
portfolio_handler = PortfolioHandler(
    config, data_handler, events_queue
)

In [15]:
strategy = MOMStrategy(config, events_queue, data_handler, portfolio_handler,
                         window = 10, num = 4, rev = 0)  

In [16]:
backtest = Backtest(config, events_queue, strategy,
                    data_handler= data_handler, portfolio_handler = portfolio_handler)

In [ ]:
results = backtest.start_trading()

---------------------------------
Running Backtest...
---------------------------------


In [ ]:
sum(results['positions']['commission_buy'])

In [ ]:
sum(results['positions']['commission_sell'])

In [ ]:
sum(results['positions']['earning'])

In [ ]:
results['cum_returns'].to_csv(config['out_dir'] + "\\cum_returns.csv")

In [ ]:
backtest.strategy.MOM_tickers